In [1]:
using OrdinaryDiffEq, ForwardDiff, Distributions, RandomNumbers, NPZ

In [ ]:
function initial_sampling(Eb,w1,mb)
    nums=rand(1)*2*pi
    x1=cos(nums[1])*sqrt(Eb/(0.5*mb*w1*w1))
    y1=sin(nums[1])*sqrt(2*mb*Eb)
    return x1,y1
end

function bolz_energy(a)
    return rand(Truncated(Exponential(1), 0., 10))
end
function bolz_freq(a)
    return rand()*0.4324555320336759+0.63
end
N=0

initial_position = [0.1, 0.,0.,0.,0.,0.,0.,0.,0.]
initial_momentum=[0. ,0.1,0.,0.,0.,0.,0.,0.,0.]
numeros=[2,4,6,8,10,16,20,26,30,36,40,50,60,80,100]
#numeros=[4,6]
promedios=100
solu=0
w=0
Es=0
count=zeros(size(numeros)[1])
for ii in 1:size(numeros)[1]
    println(ii)
    ii=Int(ii)
        
for jj in 1:promedios
    jj=Int(jj)
N=numeros[ii]
initial_position=ones(N+1)
initial_momentum=ones(N+1)

a=0.2
b=0.01

m=[1.,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
w=[0.,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
g=[0.,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]

X0=17.    
P0=1.

golden=(1+sqrt(big(5)))/2
Es=zeros(N+1)

time=10000.
#time=1.
tspan=(0., time)
#fig=figure()
w=zeros(N+1)
g=zeros(N+1)
E=zeros(N+1)
m=ones(N+1) .*0.1
m[1]=1.
w=w .* 0. .+ 1.
w=(bolz_freq.(w))
Es=(bolz_energy.(Es))



#println(w)
W=maximum(w)
g=g .* 0. .+ 0.3/N

dts=0.01*2. *pi/W
#Hsys(q,p) = p[1]^2. /(2. *m[1]) +m[1]*w[1]^2. *q[1]^2. /2. +0.5* -q[1]*g[2]*q[2]+p[2]^2. /(2. *m[2])+m[2]*w[2]^2. *q[2]^2. /2. + q[1]^2. *g[2]^2. /(2. *m[2]*w[2]^2.) -q[1]*g[3]*q[3]+p[3]^2. /(2. *m[3])+m[3]*w[3]^2. *q[3]^2. /2. + q[1]^2. *g[3]^2. /(2. *m[3]*w[3]^2.) -q[1]*g[4]*q[4]+p[4]^2. /(2. *m[4])+m[4]*w[4]^2. *q[4]^2. /2. + q[1]^2. *g[4]^2. /(2. *m[4]*w[4]^2.) -q[1]*g[5]*q[5]+p[5]^2. /(2. *m[5])+m[5]*w[5]^2. *q[5]^2. /2. + q[1]^2. *g[5]^2. /(2. *m[5]*w[5]^2.) -q[1]*g[6]*q[6]+p[6]^2. /(2. *m[6])+m[6]*w[6]^2. *q[6]^2. /2. + q[1]^2. *g[6]^2. /(2. *m[6]*w[6]^2.) -q[1]*g[7]*q[7]+p[7]^2. /(2. *m[7])+m[7]*w[7]^2. *q[7]^2. /2. + q[1]^2. *g[7]^2. /(2. *m[7]*w[7]^2.) -q[1]*g[8]*q[8]+p[8]^2. /(2. *m[8])+m[8]*w[8]^2. *q[8]^2. /2. + q[1]^2. *g[8]^2. /(2. *m[8]*w[8]^2.)-q[1]*g[9]*q[9]+p[9]^2. /(2. *m[9])+m[9]*w[9]^2. *q[9]^2. /2. + q[1]^2. *g[9]^2. /(2. *m[9]*w[9]^2.) 
function Hsys(q,p,N0)
    result=p[1]^2. /(2. *m[1]) - a*q[1]^2. /2. +b*q[1]^4. /4.
    for i in 1:N0
        result +=  -q[1]*g[i+1]*q[i+1]+p[i+1]^2. /(2. *m[2])+m[i+1]*w[i+1]^2. *q[i+1]^2. /2. + q[1]^2. *g[i+1]^2. /(2. *m[i+1]*w[i+1]^2.) 
    end
    return result
end
pdot(dp,p,q,params,t) = ForwardDiff.gradient!(dp, q->-Hsys(q, p,N), q)
qdot(dq,p,q,params,t) = ForwardDiff.gradient!(dq, p-> Hsys(q, p,N), p)



initial_position=initial_position .* 0.
initial_momentum=initial_momentum .* 0.
initial_position[1]=0.0 #xz0[jj]
initial_momentum[1]=0.0 #pz1[jj]

for i in 1:N
    i=Int(i)
    initial_position[i], initial_momentum[i]=initial_sampling(Es[i],w[i],m[i])
end

 
prob = DynamicalODEProblem(pdot, qdot, initial_momentum, initial_position, tspan)
@time sol = solve(prob, CalvoSanz4(), dt=dts)
    solu=sol

for j in 1:size(sol[N+2,:])[1]-1
    if sol[N+2,j]<0 && sol[N+2,j+1]>0
        count[ii]=count[ii]+1
    elseif sol[N+2,j]>0 && sol[N+2,j+1]<0
        count[ii]=count[ii]+1
    end
end
end

npzwrite("numeros.npy", numeros)
npzwrite("counts.npy",count)
end
println(count)



1


┌ Warning: `Truncated(d::UnivariateDistribution, l::Real, u::Real)` is deprecated, use `truncated(d, l, u)` instead.
│   caller = ip:0x0
└ @ Core :-1


 19.109884 seconds (115.97 M allocations: 3.770 GiB, 11.33% gc time)
  7.484196 seconds (90.09 M allocations: 2.583 GiB, 20.54% gc time)
  7.455676 seconds (97.14 M allocations: 2.785 GiB, 17.50% gc time)
  8.900765 seconds (104.03 M allocations: 2.982 GiB, 20.30% gc time)
  7.565725 seconds (94.74 M allocations: 2.716 GiB, 22.67% gc time)
  7.518232 seconds (94.72 M allocations: 2.715 GiB, 18.51% gc time)
  7.992038 seconds (94.79 M allocations: 2.717 GiB, 22.34% gc time)
  5.989308 seconds (81.60 M allocations: 2.339 GiB, 23.29% gc time)
  6.587247 seconds (92.16 M allocations: 2.642 GiB, 17.87% gc time)
  7.394576 seconds (98.60 M allocations: 2.827 GiB, 23.75% gc time)
  5.976392 seconds (85.46 M allocations: 2.450 GiB, 16.73% gc time)
  7.883917 seconds (92.77 M allocations: 2.659 GiB, 24.73% gc time)
  7.380654 seconds (99.52 M allocations: 2.853 GiB, 18.37% gc time)
  6.908332 seconds (84.96 M allocations: 2.436 GiB, 19.76% gc time)
  8.295984 seconds (100.90 M allocations: 2.89

 14.902260 seconds (176.83 M allocations: 5.891 GiB, 19.64% gc time)
 13.738138 seconds (175.59 M allocations: 5.849 GiB, 19.91% gc time)
 11.696240 seconds (170.64 M allocations: 5.684 GiB, 17.14% gc time)
 13.090420 seconds (171.86 M allocations: 5.725 GiB, 20.89% gc time)
 14.404352 seconds (178.65 M allocations: 5.951 GiB, 17.45% gc time)
 15.164080 seconds (186.61 M allocations: 6.217 GiB, 17.19% gc time)
 14.787867 seconds (181.49 M allocations: 6.046 GiB, 19.77% gc time)
 13.474284 seconds (167.77 M allocations: 5.589 GiB, 20.70% gc time)
 12.460101 seconds (157.05 M allocations: 5.232 GiB, 17.34% gc time)
 14.527480 seconds (170.11 M allocations: 5.667 GiB, 20.74% gc time)
 14.019265 seconds (182.53 M allocations: 6.081 GiB, 16.79% gc time)
 13.493154 seconds (165.84 M allocations: 5.525 GiB, 20.91% gc time)
 15.850764 seconds (189.51 M allocations: 6.313 GiB, 18.37% gc time)
 16.335043 seconds (184.52 M allocations: 6.147 GiB, 21.43% gc time)
 13.387283 seconds (175.72 M alloc

 19.539256 seconds (269.71 M allocations: 10.412 GiB, 17.93% gc time)
 18.746585 seconds (267.55 M allocations: 10.329 GiB, 19.16% gc time)
 19.486884 seconds (270.58 M allocations: 10.446 GiB, 18.19% gc time)
 18.717184 seconds (272.25 M allocations: 10.510 GiB, 19.21% gc time)
 18.901997 seconds (261.24 M allocations: 10.085 GiB, 20.24% gc time)
 19.953841 seconds (268.79 M allocations: 10.377 GiB, 17.42% gc time)
 17.871068 seconds (252.86 M allocations: 9.762 GiB, 18.40% gc time)
 20.791825 seconds (275.18 M allocations: 10.623 GiB, 20.88% gc time)
 19.668012 seconds (258.34 M allocations: 9.973 GiB, 20.04% gc time)
 25.127952 seconds (268.38 M allocations: 10.361 GiB, 16.12% gc time)
 22.915812 seconds (261.32 M allocations: 10.088 GiB, 25.53% gc time)
 24.502838 seconds (272.16 M allocations: 10.507 GiB, 17.96% gc time)
 22.374731 seconds (257.85 M allocations: 9.954 GiB, 17.82% gc time)
 22.986738 seconds (269.15 M allocations: 10.391 GiB, 25.06% gc time)
 19.312401 seconds (258

 27.312791 seconds (343.02 M allocations: 15.014 GiB, 16.58% gc time)
 33.170241 seconds (357.21 M allocations: 15.636 GiB, 24.10% gc time)
 28.201857 seconds (334.45 M allocations: 14.639 GiB, 20.96% gc time)
 25.231802 seconds (358.18 M allocations: 15.678 GiB, 18.57% gc time)
 27.218483 seconds (359.30 M allocations: 15.727 GiB, 16.43% gc time)
 23.511837 seconds (322.47 M allocations: 14.115 GiB, 16.38% gc time)
 26.577217 seconds (348.59 M allocations: 15.258 GiB, 17.85% gc time)
 27.377959 seconds (360.26 M allocations: 15.769 GiB, 17.97% gc time)
 26.242191 seconds (340.42 M allocations: 14.901 GiB, 15.64% gc time)
 29.168081 seconds (340.03 M allocations: 14.884 GiB, 14.98% gc time)
 29.447344 seconds (358.12 M allocations: 15.676 GiB, 24.66% gc time)
 27.850465 seconds (352.34 M allocations: 15.423 GiB, 17.97% gc time)
 25.726411 seconds (344.19 M allocations: 15.066 GiB, 15.58% gc time)
 29.644998 seconds (344.54 M allocations: 15.081 GiB, 17.90% gc time)
 28.517253 seconds (